In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [7]:
park_data = pd.read_json('Datasources/other general/parks_datasd.geojson', orient = 'records')

In [8]:
park_data

features               type
0    {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
1    {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
2    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
3    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
4    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
5    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
6    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
7    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
8    {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
9    {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
10   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
11   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
12   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
13   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
14   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
15   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
16   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
17   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
18   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
19   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
20   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
21   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
22   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
23   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
24   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
25   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
26   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
27   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
28   {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
29   {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
..                                                 ...                ...
260  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
261  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
262  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
263  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
264  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
265  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
266  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
267  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
268  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
269  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
270  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
271  {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
272  {u'geometry': {u'type': u'MultiPolygon', u'coo...  FeatureCollection
273  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
274  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
275  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
276  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
277  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
278  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
279  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
280  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
281  {u'geometry': {u'type': u'Polygon', u'coordina...  FeatureCollection
282  {u'geometry': {u'type': u'Polygon', u'coordin

In [12]:
park_data['features'][272]

{u'geometry': {u'coordinates': [[[[-117.09964458, 32.70127721],
     [-117.09952081, 32.70115748],
     [-117.10016407, 32.70116211],
     [-117.10016147, 32.70061443],
     [-117.10072968, 32.70061844],
     [-117.10075254, 32.70062621],
     [-117.10076872, 32.70063743],
     [-117.10078229, 32.70065433],
     [-117.10078409, 32.70084127],
     [-117.10076406, 32.70098757],
     [-117.10072907, 32.70111017],
     [-117.10067678, 32.7012465],
     [-117.10061846, 32.70131262],
     [-117.10053783, 32.70142874],
     [-117.10037996, 32.70159412],
     [-117.10020963, 32.70171766],
     [-117.10001776, 32.70183115],
     [-117.0999166, 32.7019021],
     [-117.09979699, 32.7020058],
     [-117.0996988, 32.70210618],
     [-117.09963001, 32.70219956],
     [-117.0995626, 32.70229633],
     [-117.09951, 32.7023998],
     [-117.09946272, 32.70249983],
     [-117.09942788, 32.70263944],
     [-117.09941092, 32.70274266],
     [-117.09939933, 32.70293082],
     [-117.09939269, 32.70321525],
 

In [10]:
park_data['type'].unique()

array([u'FeatureCollection'], dtype=object)